In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/10708\ Project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/14Gdouwq1YbxwJPoIAfI-AnPNaIq3hw2L/10708 Project


In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import re
import torch
import torch.nn as nn
from collections import Counter
import tqdm
import random
import pickle
from nltk.tokenize import wordpunct_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)


class Model(nn.Module):
    def __init__(self, embed_dim=128, gru_out=128, z_dim=1):

        super(Model, self).__init__()
        self.embed_dim = embed_dim
        self.gru_out = gru_out
        self.z_dim = z_dim

        self.gru = nn.Sequential(
            nn.Embedding(2000, embed_dim),
            nn.GRU(embed_dim, gru_out, bidirectional=True, batch_first=True),
        )

        self.classifier = nn.Linear((2*gru_out)+z_dim, 1)

    def forward(self, x, z):
        x = self.gru(x)[1].transpose(0, 1).contiguous().view((x.shape[0], -1))
        assert(x.shape == (z.shape[0], self.gru_out*2))
        x = torch.cat((x, z), dim=1)
        x = self.classifier(x)
        return x

    def forward_all_z(self, x, all_z, p_z):
        batch_size = x.size(0)
        x = self.gru(x)[1].transpose(0, 1).contiguous().view((x.shape[0], -1))

        output = torch.zeros((batch_size, 1)).cuda()
        model_outputs = []
        for p_zi, z_val in zip(p_z, all_z):
            z_val = torch.unsqueeze(z_val, 0).repeat((batch_size, 1))
            x_z = torch.cat((x, z_val), dim=1)
            model_out_z = self.classifier(x_z)
            model_out_z = torch.sigmoid(model_out_z)
            output += p_zi.item() * model_out_z
            model_outputs.append(model_out_z)

        return output, model_outputs


class BasicModel(nn.Module):
    def __init__(self, embed_dim=128, gru_out=128):

        super(BasicModel, self).__init__()
        self.embed_dim = embed_dim
        self.gru_out = gru_out

        self.gru = nn.Sequential(
            nn.Embedding(2000, embed_dim),
            nn.GRU(embed_dim, gru_out, bidirectional=True, batch_first=True),
        )

        self.classifier = nn.Linear((2*gru_out), 1)

    def forward(self, x):
        x = self.gru(x)[1].transpose(0, 1).contiguous().view((x.shape[0], -1))
        x = self.classifier(x)
        return x


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train(X_train, y_train, z_train, model, optimizer, criterion, num_epochs=1, batch_size=256, lr=0.001):
      
    model.train()
    train_size = X_train.shape[0] 

    for epoch in range(num_epochs):

        train_loss_meter = AverageMeter()
        i = 0

        for batch in range(0, train_size, batch_size):

            start_index = batch
            end_index = min(batch + batch_size, train_size)

            batch_X = X_train[start_index:end_index].cuda()
            batch_z = z_train[start_index:end_index].cuda()

            batch_y = y_train[start_index:end_index].cuda()

            output = model(batch_X, batch_z)

            loss = criterion(output, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_meter.update(loss.item(), (end_index-start_index))

            if(i % 100 == 0):
                print("Epoch: {}, Iter: {}, Training Loss: {}".format(epoch, i, train_loss_meter.avg))

            i += 1


def predict(model, X_test, p_z, batch_size=256):

    model.eval()

    with torch.no_grad():

        #all_z = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float).cuda()
        all_z = torch.tensor([[0], [1]], dtype=torch.float).cuda()

        test_size = X_test.shape[0]
        test_preds = torch.zeros((test_size, ))

        for batch in range(0, test_size, batch_size):

            start_index = batch
            end_index = min(batch + batch_size, test_size)

            batch_X = X_test[start_index:end_index].cuda()

            output, all_outputs = model.forward_all_z(batch_X, all_z, p_z)

            test_preds[start_index:end_index] = output.squeeze(dim=1).cpu()

    model.train()

    return test_preds


def train_std(X_train, y_train, model, optimizer, criterion, num_epochs=1, batch_size=256, lr=0.001):
      
    model.train()
    train_size = X_train.shape[0] 

    for epoch in range(num_epochs):

        train_loss_meter = AverageMeter()
        i = 0

        for batch in range(0, train_size, batch_size):

            start_index = batch
            end_index = min(batch + batch_size, train_size)

            batch_X = X_train[start_index:end_index].cuda()

            batch_y = y_train[start_index:end_index].cuda()

            output = model(batch_X)

            loss = criterion(output, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_meter.update(loss.item(), (end_index-start_index))

            if(i % 100 == 0):
                print("Epoch: {}, Iter: {}, Training Loss: {}".format(epoch, i, train_loss_meter.avg))

            i += 1


def predict_std(model, X_test, batch_size=256):

    model.eval()

    with torch.no_grad():

        test_size = X_test.shape[0]
        test_preds = torch.zeros((test_size, ))

        for batch in range(0, test_size, batch_size):

            start_index = batch
            end_index = min(batch + batch_size, test_size)

            batch_X = X_test[start_index:end_index].cuda()

            output = torch.sigmoid(model.forward(batch_X))

            test_preds[start_index:end_index] = output.squeeze(dim=1).cpu()

    model.train()

    return test_preds


def prepare_data(all_train_df, test_df, ind=1):

    train_df = all_train_df[ind]
    train_df = train_df.sample(frac=1)

    train_df['text'] = train_df['text'].apply(lambda x: x.lower())
    test_df['text'] = test_df['text'].apply(lambda x: x.lower())

    train_df['text'] = train_df['text'].apply(lambda x: " ".join(wordpunct_tokenize(x)))
    test_df['text'] = test_df['text'].apply(lambda x: " ".join(wordpunct_tokenize(x)))

    tokenizer = Tokenizer(num_words=2000, lower=True, split=' ', filters='#%&()*+-/:;<=>@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(train_df['text'].values)

    X_train = tokenizer.texts_to_sequences(train_df['text'].values)
    X_train = pad_sequences(X_train, maxlen=350)

    X_test = tokenizer.texts_to_sequences(test_df['text'].values)
    X_test = pad_sequences(X_test, maxlen=350)

    return X_train, X_test, train_df, test_df


def main():
    fp = open("data/train_dfs.pkl", "rb")
    all_train_df = pickle.load(fp)
    fp.close()

    fp = open("data/test_df.pkl", "rb")
    test_df = pickle.load(fp)
    fp.close()

    confounder = 'user_pop'

    for dataset_index in range(1, 10):

        print("\n-------------\nDataset Bias {}\n\n".format(dataset_index))

        X_train, X_test, train_df, test_df = prepare_data(all_train_df, test_df, dataset_index)

        X_train_tensor = torch.from_numpy(X_train)
        y_train_tensor = torch.from_numpy(train_df['label'].to_numpy()).unsqueeze(dim=1).float()
        z_train_tensor = torch.from_numpy(train_df[confounder].to_numpy()).unsqueeze(dim=1)

        X_test_tensor = torch.from_numpy(X_test)
        y_test_tensor = torch.from_numpy(test_df['label'].to_numpy()).unsqueeze(dim=1).float()
        z_test_tensor = torch.from_numpy(test_df[confounder].to_numpy()).unsqueeze(dim=1)

        num_epochs = 1
        batch_size = 256
        lr = 0.001

        model = Model().cuda()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()

        train(X_train_tensor, y_train_tensor, z_train_tensor, model, optimizer, criterion, num_epochs=1, batch_size=batch_size, lr=lr)

        p_z = Counter(train_df[confounder])
        p_z = np.array([p_z[0], p_z[1]], dtype=float)
        p_z /= p_z.sum()

        pred_test = predict(model, X_test_tensor, p_z, batch_size=batch_size)
        y_pred = np.round(pred_test.numpy())

        print("\nDataset Bias {}\n".format(dataset_index))      
        print("Accuracy: {}".format(accuracy_score(test_df['label'], y_pred)))

        model_std = BasicModel().cuda()
        optimizer_std = torch.optim.Adam(model_std.parameters(), lr=lr)

        train_std(X_train_tensor, y_train_tensor, model_std, optimizer_std, criterion, num_epochs=1, batch_size=batch_size, lr=lr)

        pred_test_std = predict_std(model_std, X_test_tensor, batch_size=batch_size)
        y_pred_std = np.round(pred_test_std.numpy())

        print("Accuracy (std): {}".format(accuracy_score(test_df['label'], y_pred_std)))

        print("\n\n-------------\n\n", dataset_index)



In [ ]:


# wrap batch of x, y, c -> loss


In [ ]:
# bi-gru model that gives feat(x)

class TextFeaturizer(nn.Module):
    def __init__(self, embed_dim=128, gru_out=32):

        super(TextFeaturizer, self).__init__()
        self.embed_dim = embed_dim
        self.gru_out = gru_out
        self.vocab_size = 2000

        self.gru = nn.Sequential(
            nn.Embedding(self.vocab_size, embed_dim),
            nn.GRU(embed_dim, gru_out, bidirectional=True, batch_first=True),
        )

        self.output_dim = 2 * gru_out

    def forward(self, x):
        batch_size = x.size(0)
        x = self.gru(x)[1].transpose(0, 1).contiguous().view((x.shape[0], -1))
        assert(x.shape == (batch_size, self.output_dim))
        return x

In [ ]:
# model that gives mu, sigma for q_phi(z|x, y, c) (inputs: x_feat, y, c)

class QPhi(nn.Module):
    def __init__(self, text_feat_dim=64, c_dim=2, z_dim=16, hidden_dim=32):

        super(TextFeaturizer, self).__init__()
        self.text_feat_dim = text_feat_dim
        self.c_dim = c_dim
        self.z_dim = z_dim
        self.y_dim = 1

        self.hidden_dim = hidden_dim

        self._model = nn.Sequential(
            nn.Linear(self.text_feat_dim + self.c_dim + self.y_dim, self.hidden_dim),
            nn.Linear(self.hidden_dim, 2 * self.z_dim)
        )

    def forward(self, x_feat, y, c):
        xcy = torch.cat([x_feat, c, y], dim=1)
        z_hat = self._model(xcy)
        z_hat_mu = z_hat[:z_hat.size(1) // 2]
        z_hat_logvar = z_hat[z_hat.size(1) // 2:]

        assert z_hat_mu.size(1) == z_hat_logvar.size(1) == self.z_dim
        return z_hat_mu, z_hat_logvar
    
    @staticmethod
    def sample(mu, logvar):
        # reparameterization
        std = torch.exp(logvar / 2.0)
        eps = torch.randn(std.shape, device=std.device)
        return mu + std * eps

In [ ]:
# model that gives p_theta(y | x, z) (inputs: feat(x), z)

class PThetaY_XZ(nn.Module):
    def __init__(self, text_feat_dim=64, z_dim=16):
        super(PThetaY_XZ, self).__init__()

        self.text_feat_dim = text_feat_dim
        self.z_dim = z_dim

        self.classifier = nn.Linear(text_feat_dim + z_dim, 1)

    def forward(self, x_feat, z):
        x = torch.cat((x_feat, z), dim=1)
        x = self.classifier(x)
        return x


In [ ]:
# model that gives p_theta(feat(x^) | z) --> 2 linear layers on top of z

class PThetaXfeat_Z(nn.Module):
    def __init__(self, z_dim=16, text_feat_dim=64, hidden_dim=32):
        super().__init__()

        self.z_dim = z_dim
        self.text_feat_dim = text_feat_dim
        self.hidden_dim = hidden_dim

        self.model = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.Linear(hidden_dim, text_feat_dim),
        )

    def forward(self, x_feat, z):
        return self.model(z)


In [ ]:
# model that gives p_theta(c | z)

class PThetaC_Z(nn.Module):
    def __init__(self, z_dim=16, c_dim=2):
        super().__init__()

        self.z_dim = z_dim
        self.c_dim = c_dim

        self.projection = nn.Linear(z_dim, c_dim)

    def forward(self, z):
        return self.projection(z)


In [ ]:
def recon_loss(x, x_hat):
    return ((x - x_hat) ** 2).mean(dim=1)

In [ ]:
# KL loss term - uses mu, sigma from q_phi, gives kl loss wrt. N(0, I)

def kl_loss(z_mu, z_logvar):
    ## VERIFY
    assert 0, "verify me"
    return - 0.5 * (1 + z_logvar - (z_mu ** 2) - z_logvar).sum() / z_mu.size(0)